# PIT - Práctica 3: Diarización de Locutores mediante Redes Neuronales "End-to-End"

**Alicia Lozano Díez**
 
6 de mayo de 2021


## Objetivo

El objetivo de esta práctica es implementar un sistema de diarización de locutores "end-to-end" basado en redes neuronales recurrentes LSTM.

### Materiales

- Guión y código (.ipynb) de la práctica - Moodle
- Datos y RTTM de entrenamiento (subconjunto de VoxConverse) * - One Drive (https://dauam-my.sharepoint.com/:u:/g/personal/alicia_lozano_uam_es/EVUnDzRYb91Gv8DkmGhLer0B0Uuvuow452szEsc0ikyIUA?download=1)
- Artículo: 
_Yusuke Fujita et al._, "End-to-End Neural Speaker Diarization with Permutation-Free Objectives", Interspeech 2019
https://www.isca-speech.org/archive/Interspeech_2019/pdfs/2899.pdf

**CUIDADO: * Los datos proporcionados son de uso exclusivo para esta práctica. No tiene permiso para copiar, distribuir o utilizar el corpus para ningún otro propósito.** 

# 1. Preparando el entorno

## 1.1. Descarga de los datos de entrenamiento

Como en las prácticas anteriores, descargaremos la lista de identificadores de los datos de entrenamiento (fichero **train_DIAR_2spk.lst** de Moodle) y los datos de entrenamiento utilizando el script **data_download_onedrive_DIAR_2spk.sh**:

In [1]:
import os 

download = True
DIR = "/content/drive/My Drive/pit/P3/"

if download and not os.path.isdir("data"):
    #from google.colab import drive
    #drive.mount("/content/drive", force_remount = True)
    %cd "$DIR"
    !cp *.lst /content/
    !cp *.sh /content
    %cd /content
    !chmod 755 *.sh
    !./data_download_onedrive_DIAR_2spk.sh

/content/drive/My Drive/pit/P3
/content
--2021-05-12 14:13:48--  https://dauam-my.sharepoint.com/:u:/g/personal/alicia_lozano_uam_es/EVUnDzRYb91Gv8DkmGhLer0B0Uuvuow452szEsc0ikyIUA?download=1
Resolving dauam-my.sharepoint.com (dauam-my.sharepoint.com)... 13.107.136.9
Connecting to dauam-my.sharepoint.com (dauam-my.sharepoint.com)|13.107.136.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/alicia_lozano_uam_es/Documents/PIT/train_2spk.zip?originalPath=aHR0cHM6Ly9kYXVhbS1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9hbGljaWFfbG96YW5vX3VhbV9lcy9FVlVuRHpSWWI5MUd2OERrbUdoTGVyMEIwVXV2dW93NDUyc3pFc2MwaWt5SVVBP3J0aW1lPU0wU0RLVkFWMlVn [following]
--2021-05-12 14:13:48--  https://dauam-my.sharepoint.com/personal/alicia_lozano_uam_es/Documents/PIT/train_2spk.zip?originalPath=aHR0cHM6Ly9kYXVhbS1teS5zaGFyZXBvaW50LmNvbS86dTovZy9wZXJzb25hbC9hbGljaWFfbG96YW5vX3VhbV9lcy9FVlVuRHpSWWI5MUd2OERrbUdoTGVyMEIwVXV2dW93NDUyc3pFc2MwaWt5SVVBP3J0aW1lPU0wU0RLVkFWMlVn
Reusing e

## 1.2. Software para evaluación del sistema

De cara a evaluar un sistema de diarización de locutores, podemos utilizar el siguiente toolbox (https://github.com/nryant/dscore):

In [2]:
!git clone https://github.com/nryant/dscore.git

Cloning into 'dscore'...
remote: Enumerating objects: 118, done.
remote: Total 118 (delta 0), reused 0 (delta 0), pack-reused 118
Receiving objects: 100% (118/118), 85.07 KiB | 6.08 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [3]:
%cd dscore
!cat requirements.txt
!pip install -r requirements.txt
%cd ..

/content/dscore
intervaltree>=3.0.0
numpy>=1.16.2
scipy>=0.17.0
tabulate>=0.5.0
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=b9b28c3e31a569ef6a826626b85cb41bb8dbc6c11464b7f53cae69a221211a02
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0
/content


Para utilizar la herramienta de scoring, utilizaríamos el siguiente comando:

```
!python score.py -r ref.rttm -s sys.rttm
```

## 1.3. Funciones auxiliares

A continuación, se definen una serie de funciones que serán útiles de cara a implementar el sistema de diarización.


* _rttm2mat_: Esta función transforma un fichero RTTM en una matriz de #frames x #speakers, con etiquetas 1 / 0 dependiendo de si dicho locutor está presente en el instante temporal (frame) dado.
Los frames correspondientes a silencio, tendrán todo 0, y se puede dar solapamiento de locutores (overlap) en cuyo caso los locutores correspondientes estarán marcados con un 1. 


In [4]:
import numpy as np

def rttm2mat(rttm_path, framerate=25, frameshift=10):
    data  = np.loadtxt(rttm_path, usecols=[3,4]) 
    spks  = np.loadtxt(rttm_path, usecols=[7],dtype='str') 
    spk_ids = np.unique(spks) 
    Ns = len(spk_ids)

    # There might be silence at the end of the file not covered by the rttm
    len_file = data[-1][0]+data[-1][1] 
    
    labels = np.zeros([int(len_file*1000),Ns]) 
    ranges = (np.array([data[:,0],data[:,0]+data[:,1]]).T*1000).astype(int) 

    for s in range(Ns): 
        for init_end in ranges[spks==spk_ids[s],:]: 
            labels[init_end[0]:init_end[1],s]=1  

    fr_labels = labels[framerate//2::frameshift,:] 
    return fr_labels

* _mat2rttm_: Esta función transforma a formato RTTM una matriz de predicciones como la indicada en la función anterior. El argumento _id_file_ es un campo del RTTM que se utiliza para identificar el fichero del que se está haciendo la diarización.

In [54]:
def mat2rttm(predictions, id_file, out_rttm, frameshift=0.01, threshold=0.5):

    hard_labels = np.zeros(predictions.shape)
    hard_labels[predictions>=threshold] = 1
    non_empty_spks = np.where(hard_labels.sum(axis=0)!=0)[0]
    hard_labels = hard_labels[:,non_empty_spks]
    
    hard_labels = np.vstack([np.zeros((1,hard_labels.shape[1])), hard_labels])

    d = np.diff(hard_labels, axis=0)
    f = open(out_rttm, 'w')
    n_spks = hard_labels.shape[1]
    for spk in range(n_spks):
        ini_indices = np.where(d[:,spk]==1)[0]
        end_indices = np.where(d[:,spk]==-1)[0]
        if (len(ini_indices) == len(end_indices) + 1):
            end_indices = np.hstack([end_indices, predictions.shape[0]])
        assert(len(ini_indices)==len(end_indices))
        n_segments = len(ini_indices)
        for index in range(n_segments):
            f.write(('SPEAKER ' + id_file + ' 1 '
                    + str(frameshift+ini_indices[index]*frameshift) + ' '
                    + str((1+end_indices[index]-ini_indices[index])*frameshift) 
                    + ' <NA> <NA> ' + 'spk' + str(spk) + ' <NA> <NA>\n'))
    f.close()

* _rttm2labs.sh_: Este script transforma un fichero RTTM en un fichero de etiquetas con formato legible en Audacity. 
Por ejemplo: "./rttm2labs.sh file1.rttm labs_file1" genera el fichero labs_file1.txt. 

#2. Implementando el sistema

##2.1. Función de coste: BCE con permutational invariant training (PIT)

Para comenzar a implementar el sistema de diarización "end-to-end", vamos a empezar por la función de coste. 

Tal y como se define en el artículo de referencia (sin tener en cuenta la parte de _deep_clustering_), la función de coste está definida como: 

<img src="https://drive.google.com/uc?export=view&id=1PPgUHSdaf861BiBTY1LOwl8a9UV9OB_4" width="300">

**PREGUNTA:**
- Con la ayuda del artículo de referencia, implemente la función _PITLoss_ a continuación, e incluya el código en la memoria de la práctica.

In [55]:
import torch
from itertools import permutations
from torch.nn.functional import binary_cross_entropy

def PITLoss(pred, target):
    """Calculate PIT loss.

    Input shape is (N, T, C), where:
        - N: batch size
        - T: sequence length
        - C: number of speakers

    Output is the average PIT loss for the whole batch. Note that
    PytTorch's built-in `binary_cross_entropy` already computes the 
    average accross all axes, so there is no need to normalize.
    """
    min_loss = np.inf
    for target_perm in permutations(target.swapaxes(0, 2)):  # Permute in speakers axis
        target_perm = torch.stack(target_perm)  # Reconstruct matrix from permuted vectors
        target_perm = target_perm.swapaxes(0, 2)  # Recover original shape
        loss = binary_cross_entropy(pred, target_perm, reduction='mean')
        if loss < min_loss:
            min_loss = loss
    return min_loss

Probamos manualmente que funciona bien en un ejemplo muy sencillo.

In [ ]:
# Ground truth labels
target = np.array([
    # First batch element (seq1)
    [[0, 0, 1, 1, 1, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 1, 1]],
    # Second batch element (seq2)
    [[1, 0, 1, 1, 1, 0, 0, 0], 
    [1, 0, 0, 0, 0, 0, 1, 1]],
], dtype=np.float64)

# Predicted labels (should actually be probabilities)
pred = np.array([
    # First batch element (seq1)
    [[0.5, 0, 1, 1, 1, 0, 0, 0], # <-- error in first component
    [0, 0, 0, 0, 0, 0, 1, 1]],
    # Second batch element (seq2)
    [[1, 0, 1, 1, 1, 0, 0, 0], 
    [1, 0, 0, 0, 0, 0, 1, 1]],
], dtype=np.float64)

# Swap the last two dimensions and convert to tensors
target = torch.tensor(np.transpose(target, (0, 2, 1)))
pred = torch.tensor(np.transpose(pred, (0, 2, 1)))

bce = PITLoss(pred, target)
print(f"Average batch loss: {bce:.4f}")

Average batch loss: 0.0217


##2.2. Definición del modelo

Ahora, vamos a definir el modelo de acuerdo al artículo: 

<img src="https://drive.google.com/uc?export=view&id=1yQ_Un-LirKOTwWG1oFxVALSztv4ZmPFx" width="400">

**PREGUNTA:**
- Defina la clase BLSTM_5layers con la configuración del modelo del artículo: El modelo tiene 5 capas BLSTM de 256 unidades cada una, y recibe características de dimensión 23. 





In [56]:
import torch
import torch.nn as nn

class BLSTM_5layers(nn.Module):
    def __init__(self, feat_dim=23, nclasses=2):
        super(BLSTM_5layers, self).__init__()
        self.blstm = nn.LSTM(feat_dim, 256, batch_first=True, 
                             bidirectional=True, num_layers=5)
        self.linear = nn.Linear(2*256, nclasses)

    def forward(self, x):
        """Input is (batch_size, seq_length, feat_dim)"""
        out = self.blstm(x)[0]
        out = self.linear(out)
        out = torch.sigmoid(out)
        return out

##2.3. Entrenamiento del sistema

Ahora vamos a completar la implementación del sistema, para realizar su entrenamiento.

Reutilizando código de prácticas anteriores, recuerda los pasos principales, y ten en cuenta las peculiaridades para esta práctica: 
- Cargar la lista de identificadores de entrenamiento. 
- Declarar el modelo y cargarlo a la GPU. 
- Declarar la función de coste (implementada previamente, PITLoss) y el optimizador. 
- En lugar de generar minibatches con una longitud fija de secuencias, puedes utilizar un tamaño de batch de _1_ y utilizar fichero a fichero.
- Para leer un audio puedes utilizar la función _read_recording_ de prácticas anteriores. 
- El tipo de parámetros o características (features) no tiene por qué ser exactamente el del artículo de referencia. Puedes utilizar por ejemplo _melspectrogram_ de la librería _librosa_.
- Puesto que las secuencias de características y las etiquetas deben tener la misma longitud, y el RTTM puede contener silencio al final, puedes realizar un _padding_ de las etiquetas a 0 hasta completar la longitud del audio (o recortar la secuencia de _features_). 

 

**PREGUNTA:**
- Completa el código siguiente e inclúyelo en la memoria.

In [8]:
import scipy.io.wavfile

def read_recording(wav_file_name): 
  fs, signal = scipy.io.wavfile.read(wav_file_name)
  signal = signal/max(abs(signal)) # normalizes amplitude
  
  return fs, signal

In [9]:
# LISTA DE ENTRENAMIENTO
TRAIN_PATH_FEATS = 'data/train_2spk/wav/'
TRAIN_PATH_LABS = 'data/train_2spk/rttm/'
with open('train_DIAR_2spk.lst', 'r') as f:
    train_list = f.read().splitlines()

In [57]:
from torch import optim
import librosa

train = True
trim_features = True
max_iters = 5
if train:
    # MODELO
    device = torch.device("cuda")
    model = BLSTM_5layers()
    model = model.to(device)

    # CRITERIOS DE OPTIMIZACIÓN
    criterion = PITLoss
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # ENTRENAMIENTO
    losses = []
    for epoch in range(1, max_iters + 1):
        model.train()
        cache_loss = 0.0

        # Batch size is 1
        for file_id in train_list:
            optimizer.zero_grad()

            # Create training sample
            fs, signal = read_recording(TRAIN_PATH_FEATS + file_id + ".wav")
            # 25ms frame length and 10ms frame shift (at fs=16000)
            feats = librosa.feature.melspectrogram(
                signal, fs, n_mels=23, win_length=400, hop_length=160)
            feats = feats.T  # n_frames x n_features

            labs = rttm2mat(TRAIN_PATH_LABS + file_id + ".rttm")  # n_frames x n_speakers
            if trim_features:
                feats = feats[:len(labs), :]
            else:
                labs = np.pad(labs, [(0, len(feats) - len(labs)), (0, 0)])

            # Make sure that all frames have defined label
            assert len(labs) == len(feats) 

            # Place data into Pytorch tensors
            feats = torch.tensor(feats[np.newaxis, :].astype("float32")).to(device)
            labs = torch.tensor(labs[np.newaxis, :].astype("float32")).to(device)
            
            # Forward the data through the network
            outputs = model(feats)

            # Compute cost
            loss = criterion(outputs, labs)

            # Backward step
            loss.backward()
            optimizer.step()
            cache_loss += loss.item()

        epoch_loss = cache_loss/len(train_list)
        losses.append(epoch_loss)
        print(f'[{epoch:02d}] loss: {epoch_loss:03f}')

        # Save model after each epoch
        if trim_features:
            torch.save(model.state_dict(), DIR + 'model_trimmed_features.pt')
        else:
            torch.save(model.state_dict(), DIR + 'model_pad.pt')

[01] loss: 0.611003
[02] loss: 0.580144
[03] loss: 0.578037
[04] loss: 0.579590
[05] loss: 0.575558


##2.4. Evaluación: DER (Diarization Error Rate)

Finalmente, realiza la evaluación de dos ficheros al azar (dentro del conjunto de entrenamiento) y obtén el DER para cada uno de ellos utilizando el toolbox _dscore_ presentado al inicio de la práctica.

**PREGUNTA:**
- Incluye en la memoria el código utilizado, los identificadores de los ficheros elegidos y sus correspondientes errores (DER).
- ¿Cómo se interpreta el DER? 
- ¿Puede tener un valor mayor de 100%?

In [10]:
# Load pre-trained model
device = torch.device("cuda")
model = BLSTM_5layers()
model.load_state_dict(torch.load(DIR + 'model.pt'))
model = model.to(device)

In [65]:
import librosa

def predict(file_id):
    model.eval()
    with torch.no_grad():
        fs, signal = read_recording(TRAIN_PATH_FEATS + file_id + ".wav")
        feats = librosa.feature.melspectrogram(
            signal, fs, n_mels=23, win_length=400, hop_length=160)
        feats = torch.tensor(feats.T[np.newaxis, :].astype("float32")).to(device)
        outputs = model(feats).cpu()[0]
    return outputs

In [21]:
for file_id in train_list:
    preds = predict(file_id)
    mat2rttm(preds, file_id, file_id + '_pred.rttm')

In [23]:
!python dscore/score.py -r data/train_2spk/rttm/*.rttm \
    -s *_pred.rttm

Loading speaker turns from reference RTTMs...
Loading speaker turns from system RTTMs...
Trimming reference speaker turns to UEM scoring regions...
Trimming system speaker turns to UEM scoring regions...
Checking for overlapping reference speaker turns...
Checking for overlapping system speaker turns...
Scoring...
File               DER    JER    B3-Precision    B3-Recall    B3-F1    GKT(ref, sys)    GKT(sys, ref)    H(ref|sys)    H(sys|ref)    MI    NMI
---------------  -----  -----  --------------  -----------  -------  ---------------  ---------------  ------------  ------------  ----  -----
akthc            25.39  61.66            0.62         1.00     0.76             1.00             0.00          1.00          0.00  0.00   0.00
bkwns            44.34  65.92            0.55         1.00     0.71             1.00             0.00          1.07          0.00  0.00   0.00
blwmj            41.27  69.87            0.49         1.00     0.66             1.00             0.00          1

Elegimos los ficheros con índices 5 y 10, correspondiente a los identificadores `djngn` y `gpjne`.

In [66]:
# Create rttm for predictions
valid_idx = [5, 10]
for idx in valid_idx:
    file_id = train_list[idx]
    preds = predict(file_id)
    mat2rttm(preds, file_id, file_id + '_pred.rttm')

In [67]:
!python dscore/score.py -r data/train_2spk/rttm/djngn.rttm \
    data/train_2spk/rttm/gpjne.rttm \
    -s djngn_pred.rttm gpjne_pred.rttm

Loading speaker turns from reference RTTMs...
Loading speaker turns from system RTTMs...
Trimming reference speaker turns to UEM scoring regions...
Trimming system speaker turns to UEM scoring regions...
Checking for overlapping reference speaker turns...
Checking for overlapping system speaker turns...
Scoring...
File               DER    JER    B3-Precision    B3-Recall    B3-F1    GKT(ref, sys)    GKT(sys, ref)    H(ref|sys)    H(sys|ref)    MI    NMI
---------------  -----  -----  --------------  -----------  -------  ---------------  ---------------  ------------  ------------  ----  -----
djngn            33.97  66.17            0.54         1.00     0.70             1.00             0.00          1.11          0.00  0.00   0.00
gpjne            24.83  61.53            0.61         1.00     0.76             1.00             0.00          1.00          0.00  0.00   0.00
*** OVERALL ***  29.15  63.85            0.57         1.00     0.73             1.00             0.40          1